In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 26 12:33:07 2018

@author: chriswintersinger
"""

import numpy as np
import pickle
import sys
#np.set_printoptions(threshold=np.nan, linewidth = 400) #Allows printing of large 17x17 numpy arrays without truncation or text wrapping.
np.set_printoptions(threshold=sys.maxsize, linewidth=400)

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile


def generate_component_dict(slat_num_binding_sites, matrix, series_key='NONAME'):
    '''
    '''
    series_dict = {}
    component_count = 1
    
    for row in matrix:
        putative_component = row[row != 0]
        mult_comp = []

        if len(putative_component) == slat_num_binding_sites: 
            series_dict.update({series_key+str(component_count):list(putative_component)})
            component_count += 1

        if ((len(putative_component)/slat_num_binding_sites).is_integer() == True) and (len(putative_component)/slat_num_binding_sites)>1: 
            for val in range(0,len(putative_component),slat_num_binding_sites):
                series_dict.update({series_key+str(component_count):list(putative_component[val:val+slat_num_binding_sites])})
                component_count += 1

        if ((len(putative_component)/slat_num_binding_sites).is_integer() == False) and (len(putative_component)/slat_num_binding_sites)>1:
            print((len(putative_component)/slat_num_binding_sites), 'off series num sites')

        #else: return False
           
    return series_dict


def convert_seq_key_to_location(std_seq_location_dict, location_dict, component_seq_key_dict, plate, well, 
                                dest=[L+str(N) for L in ['A','B','C','D','E','F','G','H'] for N in [val+1 for val in range(12)]],
                                dest_col_start=0, seq_type='NONE', dest_plate = 'dest', design_comment='FILL', vol=('FILL','FILL'), null_h2=False, diff=33):
    '''
    '''
    try:
        component_location_dict = {'node':[], 'seq':[], 'component':[], 'plate':[], 'source well':[],
                                   'destination':[], 'transfer volume':[], 'destination plate':[]}
        inc = dest_col_start
        plate_inc = 1

        transfer_vol1 = vol[0]
        transfer_vol2 = vol[1]
        
        global used_dest_wells

        for key in component_seq_key_dict.keys():
            node = 0
            
            if inc>=96: 
                plate_inc+=1
                inc=0
            
            used_dest_wells.append(dest_plate+str(plate_inc)+'-'+dest[inc])
            
            local_node_test = []
            for seq_key in component_seq_key_dict[key]: 
                #print(seq_key)
                if (seq_key == -1):
                    seq_row = std_seq_location_dict[std_seq_location_dict['ID']==(node+diff)]
                    component_location_dict['node'].append(seq_row.iloc[0]['node'])
                    component_location_dict['seq'].append('n'+str(node)+'k'+str(seq_key))
                    component_location_dict['component'].append(design_comment+'.'+key)
                    component_location_dict['plate'].append(seq_row.iloc[0]['plate'])
                    component_location_dict['source well'].append(seq_row.iloc[0]['well'])
                    component_location_dict['destination'].append(dest[inc])
                    component_location_dict['transfer volume'].append(transfer_vol2)
                    component_location_dict['destination plate'].append(dest_plate+str(plate_inc))
                    local_node_test.append(seq_row.iloc[0]['node'])
                    node += 1

                elif (seq_key != -1):
                    #seq_key = str(seq_key)
                    seq_row = location_dict[(location_dict['key']==seq_key) & (location_dict['node']==node) & (location_dict['h/ah']==seq_type)]
                    component_location_dict['seq'].append('n'+str(node)+'k'+str(seq_key))
                    component_location_dict['node'].append(seq_row.iloc[0]['node'])
                    component_location_dict['component'].append(design_comment+'.'+key)
                    component_location_dict['plate'].append(seq_row.iloc[0][plate])
                    component_location_dict['source well'].append(seq_row.iloc[0][well])
                    component_location_dict['destination'].append(dest[inc])
                    component_location_dict['transfer volume'].append(transfer_vol1)
                    component_location_dict['destination plate'].append(dest_plate+str(plate_inc))
                    local_node_test.append(seq_row.iloc[0]['node'])
                    node += 1

            if null_h2 == True:
                for val in range(32):
                    seq_row = std_seq_location_dict[std_seq_location_dict['ID']==val+1]
                    component_location_dict['seq'].append('n'+str(val)+'k'+str(seq_key)+'.H2')
                    component_location_dict['node'].append(seq_row.iloc[0]['node'])
                    component_location_dict['component'].append(design_comment+'.'+key)
                    component_location_dict['plate'].append(seq_row.iloc[0]['plate'])
                    component_location_dict['source well'].append(seq_row.iloc[0]['well'])
                    component_location_dict['destination'].append(dest[inc])
                    component_location_dict['transfer volume'].append(transfer_vol2)
                    component_location_dict['destination plate'].append(dest_plate+str(plate_inc))
                    local_node_test.append(seq_row.iloc[0]['node'])

            inc += 1

            local_node_test = sorted(local_node_test)
            test1 = sorted(list(range(32)))
            test2 = sorted(list(range(32))+list(range(32)))

            if ((local_node_test != test1) and (local_node_test != test2)):
                print("ERROR, missing nodes for slat"+key+"-", component_seq_key_dict[key],
                      '\n',local_node_test, len(local_node_test))

        return component_location_dict
    
    except:
        print('EXCEPTION ', design_comment, '-', key, '\nSEQ KEY ', seq_key, component_seq_key_dict[key])


def output_excel_sheet(component_location_dict, excel_workbook, excel_sheet_destination):
    '''
    '''
    #df = pd.DataFrame(data=component_location_dict)
    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in component_location_dict.items() ]))

    #writer = ExcelWriter(excel_workbook_destination)
    #df.to_excel(writer,excel_sheet_destination,index=False)
    
    writer = excel_workbook
    df.to_excel(writer, sheet_name=excel_sheet_destination)

## CREATE PIPETTE INSTRUCTIONS FROM EXCEL MATRIX SHEET

In [13]:
###IMPORT PREVIOUSLY CREATED MATRICES, h5 is the 7mer handle/antihandle side of the 6HB. 
###h2 is the other side where we put plug handles to seed, or other cargo of interest
x_h5 = np.array(pd.ExcelFile("output_test_for_64_sheet.xlsx").parse("x-k6-8_slats32"))
y_h5 = np.array(pd.ExcelFile("output_test_for_64_sheet.xlsx").parse("y-k6-8_slats32"))

x_h2 = np.array(pd.ExcelFile("input_example.xlsx").parse("square_x_h2_adaptors"))

In [14]:
###IMPORT SEQUENCE LOCATION DICTIONARIES
all_data=pd.ExcelFile("210208_ECHO_mapping.xlsx")
# load data from specific sheets
seq_mapping_hah = all_data.parse("echo_map_7bp_2T_32x_lib")
seq_mapping_std = all_data.parse("echo_map_6hb_std_tetrad")
seq_mapping_seed_plugs = all_data.parse("echo_map_seed_6hb_plugs_SQ_MID")

##Convert matrices of numbers into component dictionaries, where each slat gets a 1D matrix that is 32 number long.
x_comp_dict = generate_component_dict(32, x_h5,'x-')
x_h2_comp_dict = generate_component_dict(32, x_h2,'nx-')

y_comp_dict = generate_component_dict(32, y_h5.transpose(),'y-')

In [15]:
len(x_comp_dict),len(x_h2_comp_dict),len(y_comp_dict)

(32, 32, 32)

In [18]:
##VOLUMES in nL for echo below. First value is handle from library and second is for control strands with no handles.
## The first is at 200 uM per strand, vs the control at 100 uM per strand. This is why the volume of the latter is double the former.
vols=(50,100)

##Base name for the destination plate that you'll feed in the Echo. One slat per well, 96 different slats per plate.
##If more than one plate, the the base name will be incremented with an integer number. 
dest_name = 'x_64test_'

##x-slats below
used_dest_wells = []
##Null h2 is false below because there is a mix of occupied sites (i.e. the seed plug) and null strands on the x-slat h2 slat.
output_dict_1 = convert_seq_key_to_location(seq_mapping_std, seq_mapping_hah, x_comp_dict,
                                             'plate', 'well', 
                                             seq_type='h', dest_plate=dest_name, design_comment='h5x', vol=vols, 
                                             null_h2=False)
output_dict_2 = convert_seq_key_to_location(seq_mapping_std, seq_mapping_seed_plugs, x_h2_comp_dict,
                                             'plate', 'well', 
                                             seq_type='ah', dest_plate=dest_name, design_comment='h2x', vol=vols, 
                                             null_h2=False, diff=1)
##y-slats below
dest_name = 'y_64test_'
##Null h2 is true below because there is nothing appended to the y-slat h2 side.
output_dict_3 = convert_seq_key_to_location(seq_mapping_std, seq_mapping_hah, y_comp_dict,
                                             'plate', 'well', 
                                             seq_type='ah', dest_plate=dest_name, design_comment='h5y', vol=vols, 
                                             null_h2=True)

In [19]:
ds = [output_dict_1, output_dict_2, output_dict_3]
d = {'seq':[], 'component':[], 'plate':[], 'source well':[],'destination':[], 'transfer volume':[], 'destination plate':[]}

for k in d.keys():
    for d_part in ds:
        d[k]+=d_part[k]
        
d = pd.DataFrame(d)
d.rename(columns={'plate':'Source Plate Name', 'destination':'Destination Well', 
                  'destination plate':'Destination Plate Name'}, inplace=True)

d['Source Plate Type'] = '384PP_AQ_BP'

final_pipette_instruction_workbook = pd.ExcelWriter('output_pipette_instructions_for_the_echo.xlsx', engine='xlsxwriter')
output_excel_sheet(d, final_pipette_instruction_workbook, 'merged')
final_pipette_instruction_workbook.save()